In [1]:
from keras.datasets import cifar10
from keras.models import model_from_json
from keras.utils.np_utils import to_categorical
from keras.optimizers import Adam

import numpy as np
import matplotlib.pyplot as plt

from keras import backend as K

Using TensorFlow backend.


In [2]:
batch_size = 128
epochs = 5
eps = 0.01
max_n = 5000

In [3]:
num_classes = 10

In [4]:
model_name = 'models/50_model.json'
weights_name = 'weights/50_weights.h5'

In [5]:
def predict(x, model):
    pred = model.predict(np.array([x]), batch_size=1)
    pred_class = np.argmax(pred)
    pred_per = max(pred[0])
    
    return pred_class, pred_per

In [13]:
def generate_p(x, label, model_name, weights_name):
    model = model_from_json(open(model_name).read())
    model.load_weights(weights_name)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
    class_output = model.output[:, int(label)]
    grads = K.gradients(class_output, model.input)[0]
    gradient_function = K.function([model.input], [grads])
    grads_val = gradient_function([np.array([x])])
    
    K.clear_session()
    
    p = np.sign(grads_val)
    
    return p.reshape(32,32,3)

In [14]:
def generate_adv(x, label, eps, model_name, weights_name):
    p = generate_p(x, label, model_name, weights_name)
    adv = (x - eps*p).clip(min=0, max=1)
    
    return adv

In [15]:
def generate_adv_list(x_list, y_list, eps, model_name, weights_name):
    adv_list = []
    
    for i, (x, y) in enumerate(zip(x_list, y_list)):
        if i % 100 == 0:
            print(i)
            
        adv = generate_adv(x, y, eps, model_name, weights_name)
        adv_list.append(adv)
        
    return np.array(adv_list)

In [16]:
# データの読み込み
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 正規化
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

y_train_catego = to_categorical(y_train, num_classes)
y_test_catego = to_categorical(y_test, num_classes)

In [17]:
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [18]:
label =['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'boat', 'track']

In [ ]:
adv_test = generate_adv_list(x_test, y_test, eps, model_name, weights_name)
adv_test.shape

In [ ]:
model = model_from_json(open(model_name).read())
model.load_weights(weights_name)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

In [ ]:
score = model.evaluate(x_test, y_test_catego, verbose=0)
print('x')
print('Test loss :', score[0])
print('Test accuracy :', score[1])

In [ ]:
print('x')
print('p: pred_class, t: true_class')
plt.figure(figsize=(10,10))
plt.subplots_adjust(wspace=0.4, hspace=0.6)
for i in range(25):
    cifar_img=plt.subplot(5,5,i+1)
    
    img = x_test[i]
    pred_class, pred_per = predict(img, model)
    plt.imshow(img)
    plt.title('p:{0}, t:{1}'.format(label[pred_class],label[int(y_test[i])]))

plt.show()

In [ ]:
score = model.evaluate(adv_test, y_test_catego, verbose=0)
print('adv')
print('Test loss :', score[0])
print('Test accuracy :', score[1])

In [ ]:
print('adv')
print('p: pred_class, t: true_class')
plt.figure(figsize=(10,10))
plt.subplots_adjust(wspace=0.4, hspace=0.6)
for i in range(25):
    cifar_img=plt.subplot(5,5,i+1)
    
    img = adv_test[i]
    pred_class, pred_per = predict(img, model)
    plt.imshow(img)
    plt.title('p:{0}, t:{1}'.format(label[pred_class],label[int(y_test[i])]))

plt.show()

In [ ]:
adv_train = generate_adv_list(x_train[:max_n], y_train[:max_n], eps, model_name, weights_name)
adv_train.shape

In [ ]:
model = model_from_json(open(model_name).read())
model.load_weights(weights_name)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
history = model.fit(adv_train, y_train_catego[:loop*max_n],
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test_catego))
score = model.evaluate(x_test, y_test_catego, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model_json_str = model.to_json()
open('models/5000advs_model.json', 'w').write(model_json_str)
model.save_weights('weights/5000advs_weights.h5');
print('model saved.')

In [ ]:
score = model.evaluate(x_test, y_test_catego, verbose=0)
print('Test loss :', score[0])
print('Test accuracy :', score[1])

In [ ]:
score = model.evaluate(x_test[:loop*max_n], y_test_catego[:loop*max_n], verbose=0)
print('Test loss :', score[0])
print('Test accuracy :', score[1])

In [ ]:
score = model.evaluate(adv_test, y_test_catego[:loop*max_n], verbose=0)
print('Test loss :', score[0])
print('Test accuracy :', score[1])

In [ ]:
# 正解率の推移をプロット
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Accuracy')
plt.legend(['train', 'test'], loc='upper left')
# plt.show()
plt.savefig("graphs/5000adv_acc.png")

In [ ]:
plt.cla()

# ロスの推移をプロット
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.legend(['train', 'test'], loc='upper left')
# plt.show()
plt.savefig("graphs/5000adv_loss.png")